## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import os
import folium

## Read-in

In [2]:
df = pd.read_csv('multifamily distress - Sheet1.csv')

## Geocode

In [3]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [5]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['address'].apply(geocode)

In [6]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [ ]:
# df.at[index#,'col_name']

## Format Data

In [7]:
### Insert NaNs if needed ###
df = df.replace('N/A', np.nan)

## HTML Popup Formatter

In [11]:
df.head(1)

,address,status,value,lender,borrower,geocoded,lat,lon
0,"5200 North Sheridan Road, Chicago 60640",auctioned off,"$40,000,000.00",Ares Management,Spirit Invesment and The Bascom Group,"(41.97654379999999, -87.6553486)",41.976544,-87.655349


In [12]:
def popup_html(row):
    address = row['address']
    status = row['status']
    value = row['value']
    lender = row['lender']
    borrower = row['borrower']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>{}'''.format(status) + '''<br>
    <strong>Days on Market: </strong>{}'''.format(value) + '''<br>
    <strong>Listing Agent: </strong>{}'''.format(lender) + '''<br>
    <strong>Buyer's Agent: </strong>{}'''.format(borrower) + '''<br>
    </html>
    '''
    return html

In [13]:
df

,address,status,value,lender,borrower,geocoded,lat,lon
0,"5200 North Sheridan Road, Chicago 60640",auctioned off,"$40,000,000.00",Ares Management,Spirit Invesment and The Bascom Group,"(41.97654379999999, -87.6553486)",41.976544,-87.655349
1,"223 West Lake Street, Chicago 60606",auctioned off,"$6,500,000.00",Wilmington Savings Fund,Avra Properties (Manuel Gliksberg),"(41.8853654, -87.6349197)",41.885365,-87.634920
2,"2900 East 79th Street, Chicago 60649",auctioned off,"$1,452,500.00",Redwood BPL Holdings,Escanaba Investment/Carlos Perez,"(41.75204180000001, -87.5538227)",41.752042,-87.553823
3,"7752 South Racine Avenue, Chicago 60620",auctioned off,"$1,409,000.00",Wilmington Savings Fund,Racine Investment Group/Carlos Perez,"(41.7525236, -87.6541785)",41.752524,-87.654179
4,"7908 South Laflin Street, Chicago 60620",auctioned off,"$1,129,450.00",Wilmington Savings Fund,Laflin/Carlos Perez,"(41.75010169999999, -87.6611866)",41.750102,-87.661187
5,"29 South La Salle Street, Chicago 60603",foreclosure filed,"$74,000,000.00",Barings,DLC Residential,"(41.8811582, -87.63163139999999)",41.881158,-87.631631
6,"1411 South Michigan Avenue, Chicago 60605",foreclosure filed,"$80,000,000.00",Ares Management,Russland Capital,"(41.8637181, -87.623818)",41.863718,-87.623818
7,"1020 W. Lawrence Avenue, Chicago 60640",watchlisted,"$40,781,000.00",KeyBank National Association,Cedar Street Companies,"(41.969204, -87.6558063)",41.969204,-87.655806
8,"1033 W Loyola Avenue, Chicago 60626",watchlisted,"$15,750,000.00",Berkadia Commercial Mortgage,Laramar,"(42.0012661, -87.6573815)",42.001266,-87.657381


In [14]:
import folium
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Multi-family distress in Chicago')


### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    marker = folium.Marker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
        popup=folium.Popup(popup_html(row), max_width=400))
    marker.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
            
# Display map
m

In [15]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [16]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/chicago_multifamily_distress_02_05_24
